In [2]:
import numpy as np
from scipy import spatial
from scipy.ndimage import gaussian_filter, sobel
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage import data, color, img_as_float, io
from skimage.morphology import erosion, disk
import time
from math import *
from google.colab import drive
import copy
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
def get_patches(img, points, size_w):
    patches=[]
    img3=np.lib.pad(img, ((size_w//2, size_w//2),(size_w//2,size_w//2)), 'edge')
    for i in points:
        patches.append(np.array(img3[i[0]:i[0]+2*size_w//2, i[1]:i[1]+2*size_w//2].flatten()))
        
    return patches


In [4]:
def similarity(sim, patches1, patches2, maskinv, corner_pos1,corner_pos2):

    patches2 = [x*maskinv for x in patches2]
    
    patches1=patches1*maskinv
    matchall=spatial.distance.cdist(patches1, patches2, sim)
    
    match=[]
    for i in range(matchall.shape[0]):
        match.append((corner_pos1, corner_pos2[np.argmin(matchall[i])]))

    return match


In [5]:
def calc_pivot(border_I, confidence, newImg):
    for idx, i in enumerate(border_I):
        blurredpatch=get_patches(imgblur, [i], size_w)[0].reshape((size_w,size_w))
        patchmask=get_patches(masknew, [i], size_w)[0].reshape((size_w,size_w))
        dx = sobel(blurredpatch, axis=0)
        dy = -sobel(blurredpatch, axis=1)
        nx = sobel(patchmask, axis=0)
        ny = sobel(patchmask, axis=1) 

        mod = np.sqrt(dx ** 2 + dy ** 2)
        modI=(np.unravel_index(mod.argmax(), mod.shape))
        
        v1=np.array([dx[modI[0],modI[1]],dy[modI[0],modI[1]]])
        v2=np.array([ny[halfWindowSize,halfWindowSize],nx[halfWindowSize,halfWindowSize]]).T
        
        D = abs(np.dot(v1,v2/np.linalg.norm(v2)))
        D /= (sqrt(dy[modI[0], modI[1]] ** 2 + dx[modI[0], modI[1]] ** 2) *
        sqrt(nx[halfWindowSize,halfWindowSize] ** 2 + ny[halfWindowSize,halfWindowSize] ** 2))
        
        border_C[idx]=confidence[i[0],i[1]]*D
        
    indx=np.argmax(border_C)
    maxI = border_I[indx]
    return maxI


In [7]:
if __name__ == "__main__":
    img_names = "/content/gdrive/My Drive/Colab Notebooks/test_im3.bmp"
    output_path="CriminisiAlG/results/"
    output_name = ["result3.bmp"]
    mask_name = ["mask3.bmp"]
    patchSize = [11]

    for i in range(1):
        for size_w in patchSize:
            RGB_img1 = io.imread(str(img_names))
            img1 = img_as_float(color.rgb2gray(RGB_img1))

            RGB_mask = io.imread(str("/content/gdrive/My Drive/Colab Notebooks/"+mask_name[i]))
            mask = img_as_float(color.rgb2gray(RGB_mask))
            halfWindowSize = size_w // 2

            masknew=mask.copy()
            newImg = img1.copy()
            imgblur = gaussian_filter(newImg, sigma=1, order=0)
            max_x_shape, max_y_shape = newImg.shape
            start_time = time.time()
  
            patchesIndex = [(ix,iy) for ix, row in enumerate(newImg) for iy, i in enumerate(row) if (ix>size_w and iy>size_w and ix<newImg.shape[0]-size_w and iy<newImg.shape[1]-size_w and ix%halfWindowSize==0 and iy%halfWindowSize==0) and (1 not in get_patches(mask, [(ix,iy)], size_w)[0])]
            patches = get_patches(newImg, patchesIndex, size_w)

            confidence = 1. - masknew.copy()

            border = masknew - erosion(masknew, disk(1))
            border_I =  [(ix,iy) for ix, row in enumerate(border) for iy, i in enumerate(row) if i==1]  
            border_C=[0] * len(border_I)

            
            for idx, i in enumerate(border_I):
                confidence[i[0],i[1]]=sum(get_patches(confidence, [i], size_w)[0])/size_w**2
                
            while (masknew==1).sum()>0 :
                border = masknew - erosion(masknew, disk(1))
                border_I = [(ix,iy) for ix, row in enumerate(border) for iy, i in enumerate(row) if i==1 ]  
                border_C = [0] * len(border_I)
                
                pivotI = calc_pivot(border_I, confidence, newImg)
                

                pivotPatch = get_patches(newImg, [pivotI], size_w)

                maskpatch = -(get_patches(masknew, [pivotI], size_w)[0]-1)
                match = similarity('euclidean', pivotPatch, patches, maskpatch, pivotI, patchesIndex)

                pivotI_xmin=match[0][0][0]-halfWindowSize
                pivotI_xmax=match[0][0][0]+halfWindowSize
                pivotI_ymin=match[0][0][1]-halfWindowSize
                pivotI_ymax=match[0][0][1]+halfWindowSize
                
                match_xmin=match[0][1][0]-halfWindowSize
                match_xmax=match[0][1][0]+halfWindowSize
                match_ymin=match[0][1][1]-halfWindowSize
                match_ymax=match[0][1][1]+halfWindowSize

                if pivotI_xmin < 0:
                    newImg[0:pivotI_xmax, pivotI_ymin:pivotI_ymax]=newImg[match_xmin-pivotI_xmin:match_xmax, match_ymin:match_ymax]
                    masknew[0:pivotI_xmax, pivotI_ymin:pivotI_ymax]=0
                elif pivotI_ymin < 0:
                    newImg[pivotI_xmin:pivotI_xmax, 0:pivotI_ymax]=newImg[match_xmin:match_xmax, match_ymin-pivotI_ymin:match_ymax]
                    masknew[pivotI_xmin:pivotI_xmax, 0:pivotI_ymax]=0
                elif pivotI_xmax > max_x_shape:
                    newImg[pivotI_xmin:max_x_shape, pivotI_ymin:pivotI_ymax]=newImg[match_xmin:match_xmax-pivotI_xmax+max_x_shape, match_ymin:match_ymax]
                    masknew[pivotI_xmin:max_x_shape, pivotI_ymin:pivotI_ymax]=0
                elif pivotI_ymax > max_y_shape:
                    newImg[pivotI_xmin:pivotI_xmax, pivotI_ymin:pivotI_ymax]=newImg[match_xmin:match_xmax, match_ymin:match_ymax-pivotI_ymax+max_y_shape]
                    masknew[pivotI_xmin:pivotI_xmax, pivotI_ymin:pivotI_ymax]=0   
                else:
                    newImg[pivotI_xmin:pivotI_xmax, pivotI_ymin:pivotI_ymax]=newImg[match_xmin:match_xmax, match_ymin:match_ymax]
                    masknew[pivotI_xmin:pivotI_xmax, pivotI_ymin:pivotI_ymax]=0

                pivotI_xmin = match[0][0][0]-halfWindowSize
                pivotI_xmax = match[0][0][0]+halfWindowSize
                pivotI_ymin = match[0][0][1]-halfWindowSize
                pivotI_ymax = match[0][0][1]+halfWindowSize
                
                match_xmin = match[0][1][0]-halfWindowSize
                match_xmax = match[0][1][0]+halfWindowSize
                match_ymin = match[0][1][1]-halfWindowSize
                match_ymax = match[0][1][1]+halfWindowSize
                if pivotI_xmin < 0:
                    confidence[0:pivotI_xmax, pivotI_ymin:pivotI_ymax]=confidence[match_xmin-pivotI_xmin:match_xmax, match_ymin:match_ymax]
                elif pivotI_ymin < 0:
                    confidence[pivotI_xmin:pivotI_xmax, 0:pivotI_ymax]=confidence[match_xmin:match_xmax, match_ymin-pivotI_ymin:match_ymax]
                elif pivotI_xmax > max_x_shape:
                    confidence[pivotI_xmin:max_x_shape, pivotI_ymin:pivotI_ymax]=newImg[match_xmin:match_xmax-pivotI_xmax+max_x_shape, match_ymin:match_ymax]
                elif pivotI_ymax > max_y_shape:
                    confidence[pivotI_xmin:pivotI_xmax, pivotI_ymin:pivotI_ymax]=newImg[match_xmin:match_xmax, match_ymin:match_ymax-pivotI_ymax+max_y_shape]
                else:
                    confidence[pivotI_xmin:pivotI_xmax, pivotI_ymin:pivotI_ymax]=confidence[match_xmin:match_xmax, match_ymin:match_ymax]

            print("--- %s seconds ---" % (time.time() - start_time))
            io.imsave(img_names.split('.')[0]+"Criminisi.bmp", newImg)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


--- 154.3599202632904 seconds ---
